In [ ]:
#ОПИСАНИЕ столбцов

#id - идентификатор
#battery_power - емкость батареи
#blue - Bluetooth
#clock_speed - тактовая частота
#dual_sim - две SIM-карты
#four_g - поддержка 4G
#int_memory - внутренняя память
#m_dep - толщина устройства
#mobile_wt - вес мобильного устройства
#n_cores - количество ядер процессора
#px_height - разрешение по вертикали
#px_width - разрешение по горизонтали
#ram - объем оперативной памяти (RAM)
#sc_h - высота экрана
#sc_w - ширина экрана
#talk_time - время разговора
#three_g - поддержка 3G
#touch_screen - сенсорный экран
#wifi - поддержка Wi-Fi

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

from scipy.interpolate import interp1d
from scipy.interpolate import CubicSpline
from scipy.interpolate import lagrange

from numpy.polynomial.polynomial import Polynomial

df = pd.read_csv("test.csv", sep=",")
df = df.drop(columns=['pc'], axis=1)
df.head()

,id,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi
0,1,1043,1,1.8,1,14,0,5,0.1,193,3,226,1412,3476,12,7,2,0,1,0
1,2,841,1,0.5,1,4,1,61,0.8,191,5,746,857,3895,6,0,7,1,0,0
2,3,1807,1,2.8,0,1,0,27,0.9,186,3,1270,1366,2396,17,10,10,0,1,1
3,4,1546,0,0.5,1,18,1,25,0.5,96,8,295,1752,3893,10,0,7,1,1,0
4,5,1434,0,1.4,0,11,1,49,0.5,108,6,749,810,1773,15,8,7,1,0,1


In [2]:
#Линейная интерполяция

#усредняем У для одинаковых Х
df_main = df.groupby('battery_power')['clock_speed'].mean().reset_index()

#выбор данных для интерполяции
x = df_main['battery_power'][15:31].to_numpy()
y = df_main['clock_speed'][15:31].to_numpy()

#создание точек для линейной интерполяции
x_inter = np.arange(min(x), max(x)+1, 1)
y_inter = np.interp(x_inter,x,y)

fig = go.Figure()

#точки
fig.add_trace(go.Scatter(
    x = x,
    y = y,
    mode = 'markers',
    name = 'Исходные данные',
    marker=dict(color = 'blue', symbol = 'circle'))
)

# Линейная
fig.add_trace(go.Scatter(
    x = x_inter,
    y = y_inter,
    name = 'Линейная интерполяция',
    mode = 'lines',
    line = dict(color = 'red'))
)

fig.update_xaxes(title_text="Ёмкость батареи (mAh)")
fig.update_yaxes(title_text="Тактовая частота (Ghz)")
fig.update_layout(title_text='График интерполяции', title_x =0.5)
fig.show()

In [11]:
# #Интерполяци Лангранжа (неудачная)

#усредняем У для одинаковых Х
df_main = df.groupby('battery_power')['clock_speed'].mean().reset_index()

#выбор данных для интерполяции
x = df_main['battery_power'][15:31].to_numpy()
y = df_main['clock_speed'][15:31].to_numpy()

#Интерполяция Лангранжа
x_lang = np.arange(x[0],x[15]+0.01, 1)
int_poly = lagrange(x,y)
y_lang = int_poly(x_lang)

fig = go.Figure()

#точки
fig.add_trace(go.Scatter(
    x = x,
    y = y,
    mode = 'markers',
    name = 'Исходные данные',
    marker=dict(color = 'blue', symbol = 'circle'))
)

# Лангранжа
fig.add_trace(go.Scatter(
    x = x_lang,
    y = y_lang,
    mode = 'lines',
    name = 'Интерполяция Лангранжа',
    line = dict(color = 'green', shape = 'spline'))
)

fig.update_xaxes(title_text="Ёмкость батареи (mAh)")
fig.update_yaxes(title_text="Тактовая частота (Ghz)")
fig.update_layout(title_text='График интерполяции', title_x =0.5)
fig.show()

In [16]:
# #Интерполяци Лангранжа

#усредняем У для одинаковых Х
df_main = df.groupby('battery_power')['clock_speed'].mean().reset_index()

#выбор данных для интерполяции
x = df_main['battery_power'][15:31].to_numpy()
y = df_main['clock_speed'][15:31].to_numpy()

#Масштабирование данных 
x = x - np.mean(x)
x = x/np.max(x)

#y = y - np.mean(y)
#y = y/np.max(y)

#Интерполяция Лангранжа
x_lang = np.arange(x[0],x[15]+0.01, 0.01)
int_poly = lagrange(x,y)
y_lang = int_poly(x_lang)

fig = go.Figure()

#точки
fig.add_trace(go.Scatter(
    x = x,
    y = y,
    mode = 'markers',
    name = 'Исходные данные',
    marker=dict(color = 'blue', symbol = 'circle'))
)

# Лангранжа
fig.add_trace(go.Scatter(
    x = x_lang,
    y = y_lang,
    mode = 'lines',
    name = 'Интерполяция Лангранжа',
    line = dict(color = 'green', shape = 'spline'))
)

fig.update_xaxes(title_text="Ёмкость батареи (mAh)")
fig.update_yaxes(title_text="Тактовая частота (Ghz)")
fig.update_layout(title_text='График интерполяции', title_x =0.5)
fig.show()

In [17]:
#Кубический сплайн

#усредняем У для одинаковых Х
df_main = df.groupby('battery_power')['clock_speed'].mean().reset_index()

#выбор данных для интерполяции
x = df_main['battery_power'][15:31].to_numpy()
y = df_main['clock_speed'][15:31].to_numpy()

#Интерполяция Лангранжа
x_inter_lan = np.arange(min(x), max(x)+1, 1)
poly = CubicSpline(x,y)  #кубический сплайн(более точная)
y_inter_lan = poly(x_inter_lan)

fig = go.Figure()

#точки
fig.add_trace(go.Scatter(
    x = x,
    y = y,
    mode = 'markers',
    name = 'Исходные данные',
    marker=dict(color = 'blue', symbol = 'circle'))
)

# Лангранжа
fig.add_trace(go.Scatter(
    x = x_inter_lan,
    y = y_inter_lan,
    mode = 'lines',
    name = 'Интерполяция сплайн',
    line = dict(color = 'black', shape = 'spline'))
)

fig.update_xaxes(title_text="Ёмкость батареи (mAh)")
fig.update_yaxes(title_text="Тактовая частота (Ghz)")
fig.update_layout(title_text='График интерполяции', title_x =0.5)
fig.show()

In [18]:
# Интерполяция Ньютона
X_Nuton = 0
Y_Nuton = 0
#усредняем У для одинаковых Х
df_main = df.groupby('battery_power')['clock_speed'].mean().reset_index()

#выбор данных для интерполяции
x = df_main['battery_power'][15:31].to_numpy()
y = df_main['clock_speed'][15:31].to_numpy()

# Функция для вычисления разделенных разностей
def divided_differences(x, y):
    n = len(x)
    f = np.zeros((n, n))
    f[:, 0] = y
    for j in range(1, n):
        for i in range(n - j):
            f[i][j] = (f[i + 1][j - 1] - f[i][j - 1]) / (x[i + j] - x[i])
    return f[0, :]

# Вычисление интерполирующего полинома Ньютона
def newton_interpolation(x, y, x_interp):
    f = divided_differences(x, y)
    n = len(x)
    result = f[0]
    for i in range(1, n):
        term = f[i]
        for j in range(i):
            term *= (x_interp - x[j])
        result += term
    return result


X_Nuton = np.arange(min(x), max(x)+1, 1)
Y_Nuton = newton_interpolation(x,y, X_Nuton)

fig = go.Figure()

#точки
fig.add_trace(go.Scatter(
    x = x,
    y = y,
    mode = 'markers',
    name = 'Исходные данные',
    marker=dict(color = 'black', symbol = 'circle'))
)

# Интер. Ньютона график
fig.add_trace(go.Scatter(
    x = X_Nuton,
    y = Y_Nuton,
    mode = 'lines',
    name = 'Интерполяция Ньютона',
    line = dict(color = 'purple', shape = 'spline')) #shape - для сглаживания
)

fig.update_xaxes(title_text="Ёмкость батареи (mAh)")
fig.update_yaxes(title_text="Тактовая частота (Ghz)")
fig.update_layout(title_text='График интерполяции', title_x =0.5)
fig.show()


In [ ]:
#divided_differences(x, y):
#Функция вычисляет разделенные разности для заданных точек x и значений функции y.
#Разделенные разности - это специальные коэффициенты, используемые при построении интерполяционного полинома Ньютона.
#Внутри функции создается пустой массив f, где будут храниться разделенные разности. Первый столбец этого массива заполняется значениями y.
#Затем, с помощью двух вложенных циклов, вычисляются остальные значения в массиве f в соответствии с формулой разделенных разностей. 

#newton_interpolation(x, y, x_interp):
#Функция использует результаты, полученные в функции divided_differences, чтобы выполнить интерполяцию Ньютона.
#Она получает массивы x и y, которые представляют исходные данные точек.
#А также точку x_interp, в которой нужно интерполировать значение.
#Сначала функция создает массив f, вызывая divided_differences(x, y).
#Затем она использует массив f, чтобы вычислить значение интерполяции в точке x_interp.
#Для этого она применяет формулу интерполяционного полинома Ньютона, которая использует значения разделенных разностей и разницу между x_interp и точками x.

